## 📘 Pipeline de Preparação de Dados — Digital Sales Funnel Dataset

Este notebook realiza a limpeza, padronização e otimização do dataset digital_funnel_raw_dataset.csv, criando uma versão tratada e pronta para análise no Power BI.
O objetivo é estabelecer um fluxo claro de ingestão → preparação → exportação, destacando boas práticas de engenharia e análise de dados.

1. Importação do Dataset Cru

In [2]:
import pandas as pd

# Caminho do dado cru
df = pd.read_csv("../data/raw/digital_funnel_raw_dataset.csv")

# Carrega o dataset original (raw) e exibe as primeiras linhas para validação inicial dos dados.
df.head()

,fullVisitorId,visitId,date,origem,midia,dispositivo,pageviews,bounces,transactions,receita,view_product,add_to_cart,checkout,purchase
0,6722355366831429055,1486345308,20170205,(direct),(none),desktop,2.0,NaN,NaN,NaN,0,0,0,0
1,9334655840242218288,1486365162,20170205,(direct),(none),desktop,2.0,NaN,NaN,NaN,0,0,0,0
2,4410657587129722581,1486355256,20170205,(direct),(none),mobile,2.0,NaN,NaN,NaN,0,0,0,0
3,1462990062888796150,1486303509,20170205,(direct),(none),mobile,2.0,NaN,NaN,NaN,0,0,0,0
4,5007402829956156825,1486288992,20170205,(direct),(none),desktop,2.0,NaN,NaN,NaN,0,0,0,0


2. Exploração Inicial dos Dados

- Identificar:

- Tipos de dados

- Tamanho e composição do dataset

- Estatísticas descritivas

- Presença de valores ausentes

- Distribuição e consistência das variáveis

- Essa etapa fundamenta as decisões de limpeza e transformação.

In [3]:
df.info()
df.describe(include="all")
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81822 entries, 0 to 81821
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   fullVisitorId  81822 non-null  uint64 
 1   visitId        81822 non-null  int64  
 2   date           81822 non-null  int64  
 3   origem         81822 non-null  object 
 4   midia          81822 non-null  object 
 5   dispositivo    81822 non-null  object 
 6   pageviews      81819 non-null  float64
 7   bounces        40393 non-null  float64
 8   transactions   961 non-null    float64
 9   receita        961 non-null    float64
 10  view_product   81822 non-null  int64  
 11  add_to_cart    81822 non-null  int64  
 12  checkout       81822 non-null  int64  
 13  purchase       81822 non-null  int64  
dtypes: float64(4), int64(6), object(3), uint64(1)
memory usage: 8.7+ MB


fullVisitorId        0
visitId              0
date                 0
origem               0
midia                0
dispositivo          0
pageviews            3
bounces          41429
transactions     80861
receita          80861
view_product         0
add_to_cart          0
checkout             0
purchase             0
dtype: int64

2. Exploração, Agrupamento e Limpeza de Colunas

In [ ]:
df['origem'] = df['origem'].str.replace(r"[()]", "", regex=True)


# Para ver todos os valores distintos da coluna 'midia'
df['origem'].unique()

df['dispositivo'].unique()

['desktop', 'mobile', 'tablet']
Categories (3, object): ['desktop', 'mobile', 'tablet']

In [21]:
# Para ver todos os valores distintos da coluna 'midia'
df['midia'].unique()

['nenhum', 'indicacao', 'afiliado', 'cpm / display', 'organico', 'cpc / busca Paga']
Categories (6, object): ['afiliado', 'cpc / busca Paga', 'cpm / display', 'indicacao', 'nenhum', 'organico']

2.1 Renomear colunas 

In [5]:
import re

def categorizar_origem(origem):
    origem = str(origem).lower()
    if origem == 'direct':
        return 'direta'
    elif re.search(r'google|bing|yahoo|duckduckgo|baidu|yandex', origem):
        return 'organico'
    elif 'youtube' in origem:
        return 'youTube'
    elif 'facebook' in origem or 'vk.com' in origem or 't.co' in origem or 'messenger.com' in origem:
        return 'redes sociais'
    elif 'mail' in origem or 'inbox' in origem or 'outlook' in origem:
        return 'email'
    elif 'partner' in origem or 'dealspotr' in origem or 'feedly' in origem:
        return 'afiliados / parceiros'
    else:
        return 'outros sites'

df['origem'] = df['origem'].apply(categorizar_origem)

In [6]:
midia_mapping = {
    '(none)': 'nenhum',
    'affiliate': 'afiliado',
    'cpc': 'cpc / busca Paga',
    'cpm': 'cpm / display',
    'organic': 'organico',
    'referral': 'indicacao'
}

df['midia'] = df['midia'].replace(midia_mapping)

3. Padronização de Tipos de Dados

In [7]:
# Conversão da coluna data 
df['date'] = pd.to_datetime(df['date'], format="%Y%m%d")
    

3.1 Conversão de colunas categóricas

In [8]:
# As colunas origem, midia e dispositivo são tipicamente variáveis discretas. Convertê-las para category reduz o uso de memória e melhora a performance
cat_cols = ['origem', 'midia', 'dispositivo']
for col in cat_cols:
    df[col] = df[col].astype('category')

3.2 Conversão de ID para String

In [9]:
df["fullVisitorId"] = df["fullVisitorId"].astype(str)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81822 entries, 0 to 81821
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   fullVisitorId  81822 non-null  object        
 1   visitId        81822 non-null  int64         
 2   date           81822 non-null  datetime64[ns]
 3   origem         81822 non-null  category      
 4   midia          81822 non-null  category      
 5   dispositivo    81822 non-null  category      
 6   pageviews      81819 non-null  float64       
 7   bounces        40393 non-null  float64       
 8   transactions   961 non-null    float64       
 9   receita        961 non-null    float64       
 10  view_product   81822 non-null  int64         
 11  add_to_cart    81822 non-null  int64         
 12  checkout       81822 non-null  int64         
 13  purchase       81822 non-null  int64         
dtypes: category(3), datetime64[ns](1), float64(4), int64(5), object(1)
mem

4. Tratamento de Valores Ausentes

4.1 Pageviews

In [11]:
# Substitui valores ausentes por 0, pois ausência indica ausência de visualização.
df['pageviews'] = df['pageviews'].fillna(0)

4.2 Bounces

In [12]:

# Completa missing values como 0.
# Converte para inteiro.
# Interpretação: visitas sem bounce → 0.
df['bounces'] = df['bounces'].fillna(0)
df['bounces'] = df['bounces'].astype(int)

4.3 Transactions

In [13]:
# Apenas ~1% das linhas têm transações.
# Preenchimento com 0 indica ausência de compra.
df['transactions'] = df['transactions'].fillna(0)
df['transactions'] = df['transactions'].astype(int)

4.4 Receita

In [14]:
# Receita nula para visitas sem compra.
df['receita'] = df['receita'].fillna(0)

5. Otimização dos Indicadores do Funil

In [15]:
# Converter para int8 reduz drasticamente o consumo de memória 
cols_int8 = ['view_product', 'add_to_cart', 'checkout', 'purchase']

for col in cols_int8:
    df[col] = df[col].astype('int8')

6. Exportação do Dataset Preparado

- Exporta o dataset pronto para análise.

- Remove índices (index=False) para manter o CSV limpo.

- df.info() final confirma a estrutura e os tipos de dados após o processamento.

In [16]:
output_path = r"E:\Anderson\analista de dados\POWER BI\digital_funnel_project\data\processed\digital_funnel_clean.csv"
df.to_csv(output_path, index=False)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81822 entries, 0 to 81821
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   fullVisitorId  81822 non-null  object        
 1   visitId        81822 non-null  int64         
 2   date           81822 non-null  datetime64[ns]
 3   origem         81822 non-null  category      
 4   midia          81822 non-null  category      
 5   dispositivo    81822 non-null  category      
 6   pageviews      81822 non-null  float64       
 7   bounces        81822 non-null  int64         
 8   transactions   81822 non-null  int64         
 9   receita        81822 non-null  float64       
 10  view_product   81822 non-null  int8          
 11  add_to_cart    81822 non-null  int8          
 12  checkout       81822 non-null  int8          
 13  purchase       81822 non-null  int8          
dtypes: category(3), datetime64[ns](1), float64(2), int64(3), int8(4), obje